In [1]:
from copy import deepcopy
import nltk
import numpy as np
import MultiTree as MT

In [4]:
MT.tuple_list

[([u'S', u'T', u'AY', u'N', u'B', u'ER', u'G', u'ER'], [u'steinberger']),
 ([u'B', u'EH', u'L', u'V', u'Y', u'UW'], [u'bellevue']),
 ([u'S', u'W', u'IH', u'N', u'T', u'AH', u'N'], [u'swinton']),
 ([u'AE', u'M', u'D', u'EH', u'K'], [u'amdek', u'amdec']),
 ([u'B', u'AA', u'D', u'IY', u'Z'], [u'bodies', u"body's"]),
 ([u'HH', u'AE', u'K', u'AH', u'N'], [u'hakon']),
 ([u'OW', u'P', u'EH', u'K'], [u'opec']),
 ([u'OW', u'P', u'EH', u'R'], [u'opere']),
 ([u'JH', u'EH', u'F', u'R', u'IY', u'Z'],
  [u'jeffries', u"jeffrey's", u"geoffrey's", u'jeffreys', u'jefferies']),
 ([u'M', u'EH', u'N', u'AH', u'N', u'AY', u'T', u'S'], [u'mennonites']),
 ([u'P', u'AY', u'N', u'T', u'ER'], [u'pinter']),
 ([u'AH', u'N', u'IH', u'SH', u'UW', u'D'], [u'unissued']),
 ([u'L', u'AO', u'D', u'Z'], [u'lauds']),
 ([u'B', u'EH', u'CH', u'ER'], [u'betcher']),
 ([u'AA', u'TH', u'M', u'AH', u'N'], [u'othman']),
 ([u'P', u'R', u'IH', u'D', u'IH', u'K', u'AH', u'M', u'AH', u'N', u'T'],
  [u'predicament']),
 ([u'M', u'AE', 

In [3]:
class DecodeInfo:
    def __init__(self, w_h = [], p_w = [], orig = None):
        if not orig:
            self.word_hist = deepcopy(w_h)
            self.partial_word = deepcopy(p_w)
            self.tree = MT.search_tree
        else:
            self.word_hist = deepcopy(orig.word_hist)
            self.partial_word = deepcopy(orig.partial_word)
            self.tree = orig.tree
    
    def __hash__(self):
        return hash(" ".join(self.word_hist) + "|" + " ".join(self.partial_word))

    def __eq__(self, other):
        return (self.word_hist, self.partial_word) == (other.word_hist, other.partial_word)
    
    def empty(self):
        return not (self.word_hist or self.partial_word)
    
    def last_phoneme(self):
        if not self.partial_word:
            return partial_word[-1]
        
        elif not self.word_hist:
            #gives last phoneme of last word
            last_word = self.word_hist[-1]
            
            #arpabet is word to phoneme dictionary, stresses removed
            return MT.arpabet[last_word][-1]
        else:
            return None
            
    def one_fewer(self):
        if not self.partial_word:
            return DecodeInfo(w_h = self.word_hist, p_w = self.partial_word[:-1])
        elif not self.word_hist:
            last_word = self.word_hist[-1]
            last_phons = MT.arpabet[last_word]
            return DecodeInfo(w_h = self.word_hist[:-1], p_w = self.partial_word[:-1])
        else:
            return None
    
    def add_phon(self, new_phon):
        self.partial_word.append(new_phon)
        self.tree = self.tree.lookup([new_phon])
        if self.tree == None:
            return False
        
        else:
            return True
    

In [5]:
def get_best_paths(B, pr_b, pr_nb, width, time):
    def get_prob(y):
        return pr_b[time][y] + pr_nb[time][y]
    
    return sorted(B, key=get_prob, reverse=True)[:width]


def beam_search(T, p_ctc, p_lang, width, beta):

    pr_nb = [{}]*(T+1)
    pr_b = [{}]*(T+1)
    
    #extension probability of adding a phoneme k
    def p_ext(k, y, t):
        p_last = pr_nb[t-1][y]
        if y.last() != k:
            p_last += pr_b[t-1][y]
        return times([p_ctc(k,t), power(p_trans(k, y), beta), p_last])
    
    #derived phoneme transition prob from language model
    def p_trans(k, y):
        if y.tree.lookup([k]) == None:
            return 0.
        
        denom = np.sum(np.asarray([p_lang(x, y) for x in y.tree.get_leaves()]))
        numer = np.sum(np.asarray([p_lang(x, y) for x in y.tree.lookup([k]).get_leaves()]))
        return divide(numer, denom)
    
    #penalty probability for converting to a homophone
    def p_homophone(word, y):
        denom = np.sum(np.asarray([p_lang(x, y) for x in y.tree.get_leaves()]))
        numer = p_lang(word, y)
        return divide(numer, denom)
    #initialising non-blank prob
    pr_nb[0][DecodeInfo()] = 1
    
    #most probable sequences
    B = [DecodeInfo()]
    
    for t in range(1, T+1):
        B_hat = get_best_paths(B, pr_b, pr_nb, width, t-1)
        B = []
        for y in B_hat:
            if not y.empty():
                pr_nb[t][y] = times([pr_nb[t-1][y], p_ctc(y.last_phoneme(), t)])
                if y.one_fewer() in B_hat:
                    pr_nb[t][y] += p_trans(y.last_phoneme(), y.one_fewer(), t)
            
            pr_b[t][y] = times(pr_b[t-1][y], p_ctc("-", t))
            
            B.append(y)
            for k in MT.phonemes:
                
                #if adding a phoneme k is valid
                if k in y.tree.branches:
                    
                    pr_plus = p_trans(k, y, t)
                    
                    #initialising y_new = y+k
                    y_new = DecodeInfo(orig = y)
                    y_new.add_phoneme(k)
                    
                    conversion_penalty_prob = 0.
                    #handles conversion from phoneme string to word
                    for word in y_new.tree.leaf:
                        
                        #converted_y converts phoneme part of y_new into a new word
                        converted_y = DecodeInfo()
                        converted_y.word_hist = deepcopy(y_new.word_hist).append(word)
                        
                        pr_b[t][converted_y] = 0.
                        pr_nb[t][converted_y] = times(pr_plus, power(p_homophone(word, y_new), beta))
                        B.append(converted_y)
                        conversion_penalty_prob += p_homophone(word, y_new)

                    #penalises non-converted string
                    pr_nb[t][y_new] = times(pr_plus, power(conversion_penalty_prob, beta))
                    B.append(y_new)

    return get_best_paths(B, pr_b, pr_nb, 1)
    

In [42]:
def beam_search():
    # Iterate over input timesteps
    
    
    for t in range(1, T+1):

        # Consider all of the (w,p) pairs which were candidates given data up to time t-1
        # and compute the probabilities of 'some' new strings given data up to time t
        # 'some' = the current strings and 1-phoneme extensions of them
        for (w,p) in Z[t-1]:
            
            # We need to compute:
            
            # 1. p_nb[t][(w,p)]
            # 2. p_b[t][(w,p)]
            # 3. p_nb[t][(w,p + k)] for all k not blank. Remembering that this may lead to a new word.
           
            # Consider the extension by a blank phoneme
            p_nb[t][(w,p)] = p_nb[t-1][(w,p)] + char_probs[t, alphabet[(w,p)[-1]]] if (w,p)!=("","") else None # plus some other stuff???
            p_b[t][(w,p)] =  p[t-1][(w,p)] + char_probs[t, alphabet["-"]]
        
            # Consider extensions of the uncontracted thing by one phoneme
            for k in phons[:-1]:
                
                # When the extension definitely adds a phoneme to the contraction
                # all the probs are nb because we aren't adding blank!
                if k!="-" and k != (w,p)[-1]:

                    # if p+k can be read as a word v, compute the probability of (wv,0) using the lm
                    # for every such v
                    # Note that (-p_w_p_star[(w,p)])*alpha exactly cancels the same term in a prev computation of p[t-1][(w,p)]
                    # This is because (unlogarithmed) p[t-1] can be factored into acoustic and language factors
                    # where exp(p_w_p_star[(w,p)]*alpha) is the language factor
                    # The new language model factor is just the lm evaluated on v and the sentence
                    for h in phonemes_to_homophones(p+" "+k):
                        p_nb[t][(w+" "+h, "")] = char_probs[t, alphabet[k]] \
                                                    + p[t-1][(w,p)]   \
                                                    + (lm(h, w)-p_w_p_star[(w,p)]) * alpha
                        
                        p_b[t][(w+" "+h, "")] =  None
                              
                    # Compute the probability of (w,p+k) under the language model, NOT CONSIDERING THE ACOUSTIC MODEL
                    # This is the sum of probabilities of wx where p is a strict prefix of every x
                    p_w_p_star[(w, p+k)] = get_p_w_p_star(w, p+k)
                            
                    p_nb[t][(w,p+k)] = char_probs[t, alphabet[k]] \
                                        + p[t-1][(w,p)]   \
                                        + (p_w_p_star[(w,p+k)]-p_w_p_star[(w,p)]) * alpha
                            
                # When extension is the same as the final phoneme
                # the extension only extends y if the final phoneme in the uncontracted thing is blank
                elif k!="-" and k==(w,p)[-1]:
                    p_nb[t][(w,p+k)] = char_probs[t, alphabet[k]] \
                                        + p_b[t-1][(w,p)] \
                                        + (p_w_p_star[(w,p+k)]-p_w_p_star[(w,p)]) * alpha 
                else: # k == "-"
                    continue
                
            # Set the probability of a string given input data up to time t as the sum of the marginals over t'th model out
            p[t] = {y: np.log(np.exp(p_b[t][y]) + np.exp(p_nb[t][y])) for y in p_b[t].keys()}
            # Take the top k strings 
            Z[t] = top_keys(p[t], n_paths, beta)
    return Z[T]

True

In [32]:
nltk.corpus.cmudict.dict()["today"]

[[u'T', u'AH0', u'D', u'EY1'], [u'T', u'UW0', u'D', u'EY1']]

In [44]:
test_dict = {}
list_ = [test_dict]
test_dict[1] = 2
list_

[{1: 2}]

In [1]:
b = [1,2,3]

In [2]:
a = b
a[1] = 5
b

[1, 5, 3]